### Bibliothèques nécessaires

In [1]:
import pandas as pd
import numpy as np

#### Importer les données dans un DataFrame Pandas

In [2]:
df = pd.read_excel("data.xlsx")

### 2. Apprentissage du dictionnaire D avec la méthode K-SVD

In [3]:
df.shape

(2430, 150)

In [4]:
df.dropna(inplace=True)

In [5]:
df.shape

(98, 150)

In [6]:
def omp(X, D, eps, N):
    m, n = D.shape
    R = X
    alpha = np.zeros(n)
    K = 0
    P = []
    res = []
    while np.linalg.norm(R,2)>eps and K<N:
        for i in range(n):
            d = D[:,i]
            norm_d = np.linalg.norm(d,2)
            B = abs(np.transpose(d)@R)
            res.append(B/norm_d)
        M = np.argmax(res)
        P.append(M)
        phi= D[:,P]
        alpha[P] = np.linalg.pinv(phi)@X
        R = X-D@alpha
        res = []
        K = K+1
    return [alpha, R]

In [7]:
def ksvd(X, K, eps, N):
    m, n = X.shape
    R = np.zeros((N, n)) # Une matrice où je veux mettre les normes des résidus afin de voir le comportement de la méthode
    MAX_ITR = int(np.round(K/10)) # le nombre d'iteration maximal à effectuer pour l'OMP
    D0 = X[:, :K] # On initialise le dictionnaire en considérant les premières colonnes des veceturs d'apprentissage
    s = np.sqrt(np.diag(D0.T @ D0))
    for i in range(K):
        D0[:, i] = D0[:, i] / s[i] # On normalise les colonnes du dictionnaire
    Alpha = np.zeros((K, n)) # On cherche n solutions parcimonieuses pour les vecteurs d'apprentissage, les alpha_i sont de dimension égale à K
    for j in range(N):
        for i_vect in range(n):
            Alpha[:, i_vect], r = omp(X[:, i_vect], D0, eps, MAX_ITR) #On effectue une OMP et on renvoie la solution parcimonieuse, j'ai choisi de renvoyer également les normes des résidus
            R[j, i_vect] = np.linalg.norm(r)
        D = D0
        for i_col in range(K): # Effectuer une SVD sur chaque colonne==> K colonnes issues de SVD
            idx_k = np.nonzero(Alpha[i_col, :] != 0)[0]
            if len(idx_k) > 0:
                E_k = X - D @ Alpha + np.outer(D[:, i_col], Alpha[i_col, :])
                Omega = np.zeros((n, len(idx_k)))
                for inz in range(len(idx_k)):
                    Omega[idx_k[inz], inz] = 1
                E_kR = E_k @ Omega
                U, delta, V = np.linalg.svd(E_kR)
                D0[:, i_col] = U[:, 0]
                Alpha[i_col, idx_k] = delta[0] * V[0, :]
            else:
                g = np.random.randint(0, n)
                D0[:, i_col] = X[:, g] / np.linalg.norm(X[:, g])
    return D0, Alpha, R


In [8]:
# Définir les paramètres de l'apprentissage du dictionnaire
eps=1e-6
K = 150
N = 100
X = df.to_numpy()
D, Alpha, r =ksvd(X, K, eps, N)

In [9]:
D.shape

(98, 150)

In [12]:
D[0,0]

0.9866278570337588

### 3. La méthode StOMP

In [10]:
def StOMP(x, D, k, t, eps):
    """
    Approxime y par une combinaison linéaire de k colonnes de D en utilisant l'algorithme StOMP
    x: signal d'entrée
    D: dictionnaire
    k: contrainte de parcimonie
    t: dans [2,3]
    eps: critère d'arrêt
    return: approximation parcimonieuse Alpha
    """
    n = D.shape[1] # nombre d'atomes dans le dictionnaire
    Alpha = np.zeros(n) # initialisation de l'approximation à zéro
    r = x # initialisation du résidu à x
    omega = [] # ensemble des indices des atomes sélectionnés
    Cj = []
    while np.linalg.norm(r) > eps: # boucle principale de l'algorithme
        for j in range(n):
            # Calcul du vecteur des contributions de tous les atomes
            Cj.append(np.abs(D[:,j].T @ r)/np.linalg.norm(D[:,j]))
            print(f"Cj = {Cj}")
            # Calcule le seuillage
            Sk = t*np.linalg.norm(r)/np.sqrt(n)
            print(f"Sk = {Sk}")
            # Indices des atomes dont la contribution est supérieure au seuillage Sk
            Ak = list(np.where(Cj>Sk)[0])
            print(f"Ak = {Ak}")
            # On met à jour l’ensemble des indices
            omega.extend(Ak)
            omega = list(set(omega)) # On élimine les doublons pour éviter la redondance
            print(f"omega = {omega}")
            # On construit la matrice Phi
            Phi_j = [D[:,p] for p in omega]
            print(f"Taille de Phi_j = {np.shape(Phi_j)}")
            print(f"Taille de r = {np.shape(np.atleast_2d(r).T)}")
            # Calculer Alpha[j] par la méthode des moindres carrés
            Alpha_j, resid, rank, s = np.linalg.lstsq(Phi_j, np.atleast_2d(r).T, rcond=None)
    
            # Mettre à jour le vecteur Alpha et le résidu r
            Alpha[j] = Alpha_j[0]
            r = r - Phi_j * Alpha_j
    return Alpha

In [11]:
StOMP(X, D, 10, 2, 1e-6)

Cj = [array([1.        , 0.1542519 , 0.15643539, 0.32690762, 0.43644314,
       0.28291246, 0.2682482 , 0.15164881, 0.14996635, 0.1605412 ,
       0.42380472, 0.26227401, 0.20723458, 0.15819453, 0.27665349,
       0.26548752, 0.23954435, 0.26839508, 0.31455035, 0.23958802,
       0.26296407, 0.27727936, 0.13407415, 0.23237489, 0.2601079 ,
       0.27222071, 0.26123383, 0.23970003, 0.2397306 , 0.21621513,
       0.25084097, 0.22913544, 0.12794649, 0.22831751, 0.24520053,
       0.2576121 , 0.20763822, 0.25200529, 0.25617892, 0.26823994,
       0.25039503, 0.25914602, 0.27807476, 0.44989401, 0.2539329 ,
       0.18060083, 0.49498376, 0.15317473, 0.23664894, 0.25248562,
       0.24755425, 0.21966198, 0.24464411, 0.25618794, 0.23039821,
       0.25382143, 0.25359141, 0.25460895, 0.10080259, 0.24036696,
       0.2243238 , 0.25831816, 0.2425527 , 0.25534864, 0.25042927,
       0.26242328, 0.26435058, 0.23888823, 0.17164536, 0.26418872,
       0.26051196, 0.24880396, 0.26307665, 0.24701374, 0

LinAlgError: 1-dimensional array given. Array must be two-dimensional